<a href="https://colab.research.google.com/github/fvirtuozo/ML_for_Project_Time_Prediction/blob/main/Scripts_for_portifolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
import pandas as pd

import pandas as pd
import re
import os
import tabulate
import time
import numpy as np
import random
import string
from faker import Faker

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_log_error, r2_score


from google.colab import drive

from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default
import gspread

Need to Prepare in this project:


1.   Merge Datasets Asist Origin and Asist Open (data from ticket manager separates in 2 files)
2.   Convert time in seconds to Days
3. Extract the Project Number from the Jira Summary (worklogs and projects) - this will link the table Jira with the table Asist
4. Add the categorization of who is Developer and who is Tester, and from which project each belongs
5. Convert DateTime formats to  format='%d/%m/%Y'
6. Select Columns to be Used



Goal of the analysis:

1. Create a pivot table that show the Classification of projects/TMA depending on their original estimate time (+ than 5 days is a TMA)
2. Create a table that shows per person and per week how many days were logged in Jira
3. Create a table with the non facturable days
4. Show per project: Time Spent on Development, Original estimate of project, Time of development spent on Bugs opened by Staci, time of development spent on bugs opened by HN, Time spent on Testes vs original estimate, Number of bugs opened by HN and Staci, AVG solution time
5. Apply ML to predict how many time to solve new tickets and a classification model by Priority or urgent

#Importing Data Sources and Creating data Output

In [230]:
drive.mount('/content/drive')
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [231]:
tabela_asist = '1sQnSVJQWP3-MsJLjUDW8qv_gr2EGJd1fMa_awsJSf34'
tabela_jira = '1tOvC8lU3UNnt_9-pUVkAPjBw4XzVN7_1lqjclexV3C4'
tabela_worklogs = '1OVKN6RaT2XM-Miy5S33RLCaZn9S9FnQ6LG2TYNdimmc'

In [232]:
asist_output = gc.open_by_key(tabela_asist).sheet1
jira_output = gc.open_by_key(tabela_jira).sheet1
worklogs_output = gc.open_by_key(tabela_worklogs).sheet1

In [233]:
worklogs = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Portifolio/For Upload/worklogs_2025.xlsx")

worklogs_edited = gc.open_by_key(tabela_worklogs).sheet1
worklogs_edited = pd.DataFrame(worklogs_edited.get_all_records())

db_asist_open = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Portifolio/For Upload/tickets_open.xlsx', header = None)
db_asist_origin = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Portifolio/For Upload/tickets_closed.xlsx", header = None)

df_Projects = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Portifolio/For Upload/JiraExport.xlsx')
folder_to_save = r'.\Data_Sources'

In [234]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Exploration of data - Project's Database

In [235]:
df_Projects

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Parent,Project name,Assignee,Creator,Created,Updated,Σ Original Estimate,Σ Time Spent
0,TICKET-735 Help record his.,ISSUE-308,7228,Feature,Resolved,PROJECT-833,NaN,"Rodriguez, Nash and Vargas",Bryan Cobb,Aaron Levy,2025-02-14,2025-02-06,8,34
1,TICKET-761 Though suggest Mrs indeed pressure ...,ISSUE-911,5822,Improvement,Resolved,PROJECT-179,NaN,"Odom, Phillips and Vang",Adam Ho,Catherine Monroe,2025-01-23,2025-01-25,22,6
2,TICKET-889 Various color and quite investment ...,ISSUE-318,7829,Bug,Closed,PROJECT-000,ISSUE-491,Miller LLC,Shelley Pollard,Kelsey Harris,2025-02-06,2025-02-11,88,31
3,TICKET-442 Charge law new simple science parti...,ISSUE-554,3230,Bug,Open,PROJECT-855,ISSUE-866,Acosta PLC,Sheryl Williams,Charlene Watson,2025-02-11,2025-02-09,75,21
4,TICKET-764 Necessary brother believe boy.,ISSUE-149,8526,Improvement,Closed,PROJECT-060,ISSUE-962,White-Patterson,John Pitts,Beth Hull,2025-01-24,2025-01-30,86,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,TICKET-951 Professional charge commercial itself.,ISSUE-497,7713,Improvement,Closed,PROJECT-655,ISSUE-136,Newman-Sandoval,Kristin Ramirez,William Randolph,2025-03-17,2025-02-07,74,9
196,TICKET-852 Evidence true quality.,ISSUE-888,4250,Bug,Closed,PROJECT-066,NaN,"Baldwin, Barnett and Koch",Glen Ortiz,Elizabeth Harris,2025-01-24,2025-01-05,19,11
197,TICKET-270 If likely base sound how although s...,ISSUE-558,1689,Bug,Resolved,PROJECT-639,NaN,"Calderon, Clark and Fox",Shaun Myers,Ronald Hoffman,2025-01-12,2025-01-14,94,2
198,TICKET-119 General listen democratic tonight f...,ISSUE-699,9382,Improvement,Open,PROJECT-552,NaN,Martinez Inc,Michele Zimmerman,Samuel Baxter,2025-01-25,2025-03-17,30,21


In [236]:
colunas = [
    'Summary',
    'Issue key',
    'Issue id',
    'Issue Type',
    'Status',
    'Project key',
    'Parent',
    'Project name',
    'Assignee',
    'Creator',
    'Created',
    'Updated',
    'Σ Original Estimate',
    'Σ Time Spent',
]

In [237]:
#Selecting Columns
df_Projects = df_Projects[colunas]
df_Projects

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Parent,Project name,Assignee,Creator,Created,Updated,Σ Original Estimate,Σ Time Spent
0,TICKET-735 Help record his.,ISSUE-308,7228,Feature,Resolved,PROJECT-833,NaN,"Rodriguez, Nash and Vargas",Bryan Cobb,Aaron Levy,2025-02-14,2025-02-06,8,34
1,TICKET-761 Though suggest Mrs indeed pressure ...,ISSUE-911,5822,Improvement,Resolved,PROJECT-179,NaN,"Odom, Phillips and Vang",Adam Ho,Catherine Monroe,2025-01-23,2025-01-25,22,6
2,TICKET-889 Various color and quite investment ...,ISSUE-318,7829,Bug,Closed,PROJECT-000,ISSUE-491,Miller LLC,Shelley Pollard,Kelsey Harris,2025-02-06,2025-02-11,88,31
3,TICKET-442 Charge law new simple science parti...,ISSUE-554,3230,Bug,Open,PROJECT-855,ISSUE-866,Acosta PLC,Sheryl Williams,Charlene Watson,2025-02-11,2025-02-09,75,21
4,TICKET-764 Necessary brother believe boy.,ISSUE-149,8526,Improvement,Closed,PROJECT-060,ISSUE-962,White-Patterson,John Pitts,Beth Hull,2025-01-24,2025-01-30,86,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,TICKET-951 Professional charge commercial itself.,ISSUE-497,7713,Improvement,Closed,PROJECT-655,ISSUE-136,Newman-Sandoval,Kristin Ramirez,William Randolph,2025-03-17,2025-02-07,74,9
196,TICKET-852 Evidence true quality.,ISSUE-888,4250,Bug,Closed,PROJECT-066,NaN,"Baldwin, Barnett and Koch",Glen Ortiz,Elizabeth Harris,2025-01-24,2025-01-05,19,11
197,TICKET-270 If likely base sound how although s...,ISSUE-558,1689,Bug,Resolved,PROJECT-639,NaN,"Calderon, Clark and Fox",Shaun Myers,Ronald Hoffman,2025-01-12,2025-01-14,94,2
198,TICKET-119 General listen democratic tonight f...,ISSUE-699,9382,Improvement,Open,PROJECT-552,NaN,Martinez Inc,Michele Zimmerman,Samuel Baxter,2025-01-25,2025-03-17,30,21


In [238]:
df_Projects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Summary              200 non-null    object        
 1   Issue key            200 non-null    object        
 2   Issue id             200 non-null    int64         
 3   Issue Type           200 non-null    object        
 4   Status               200 non-null    object        
 5   Project key          200 non-null    object        
 6   Parent               110 non-null    object        
 7   Project name         200 non-null    object        
 8   Assignee             200 non-null    object        
 9   Creator              200 non-null    object        
 10  Created              200 non-null    datetime64[ns]
 11  Updated              200 non-null    datetime64[ns]
 12  Σ Original Estimate  200 non-null    int64         
 13  Σ Time Spent         200 non-null  

In [239]:
#Changing data types
df_Projects['Created'] = pd.to_datetime(df_Projects['Created'])
df_Projects['Updated'] = pd.to_datetime(df_Projects['Updated'])
df_Projects['Issue id'] = df_Projects['Issue id'].astype(object)
df_Projects['Parent'] = df_Projects['Parent'].astype(object)

In [240]:
#Replacing NA Values
df_Projects['Σ Original Estimate'] = df_Projects['Σ Original Estimate'].fillna(0)
df_Projects['Σ Time Spent'] = df_Projects['Σ Time Spent'].fillna(0)
df_Projects['Parent'] = df_Projects['Parent'].fillna('N/A')
df_Projects[ 'Assignee'] = df_Projects['Assignee'].fillna('N/A')

In [241]:
df_Projects

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Parent,Project name,Assignee,Creator,Created,Updated,Σ Original Estimate,Σ Time Spent
0,TICKET-735 Help record his.,ISSUE-308,7228,Feature,Resolved,PROJECT-833,N/A,"Rodriguez, Nash and Vargas",Bryan Cobb,Aaron Levy,2025-02-14,2025-02-06,8,34
1,TICKET-761 Though suggest Mrs indeed pressure ...,ISSUE-911,5822,Improvement,Resolved,PROJECT-179,N/A,"Odom, Phillips and Vang",Adam Ho,Catherine Monroe,2025-01-23,2025-01-25,22,6
2,TICKET-889 Various color and quite investment ...,ISSUE-318,7829,Bug,Closed,PROJECT-000,ISSUE-491,Miller LLC,Shelley Pollard,Kelsey Harris,2025-02-06,2025-02-11,88,31
3,TICKET-442 Charge law new simple science parti...,ISSUE-554,3230,Bug,Open,PROJECT-855,ISSUE-866,Acosta PLC,Sheryl Williams,Charlene Watson,2025-02-11,2025-02-09,75,21
4,TICKET-764 Necessary brother believe boy.,ISSUE-149,8526,Improvement,Closed,PROJECT-060,ISSUE-962,White-Patterson,John Pitts,Beth Hull,2025-01-24,2025-01-30,86,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,TICKET-951 Professional charge commercial itself.,ISSUE-497,7713,Improvement,Closed,PROJECT-655,ISSUE-136,Newman-Sandoval,Kristin Ramirez,William Randolph,2025-03-17,2025-02-07,74,9
196,TICKET-852 Evidence true quality.,ISSUE-888,4250,Bug,Closed,PROJECT-066,N/A,"Baldwin, Barnett and Koch",Glen Ortiz,Elizabeth Harris,2025-01-24,2025-01-05,19,11
197,TICKET-270 If likely base sound how although s...,ISSUE-558,1689,Bug,Resolved,PROJECT-639,N/A,"Calderon, Clark and Fox",Shaun Myers,Ronald Hoffman,2025-01-12,2025-01-14,94,2
198,TICKET-119 General listen democratic tonight f...,ISSUE-699,9382,Improvement,Open,PROJECT-552,N/A,Martinez Inc,Michele Zimmerman,Samuel Baxter,2025-01-25,2025-03-17,30,21


In [242]:
df_Projects['Σ Original Estimate'] = (df_Projects['Σ Original Estimate']/28800).round(2)
df_Projects['Σ Time Spent'] = (df_Projects['Σ Time Spent']/28800).round(2)


In [243]:
#Extracting the Asist Project Number
def ticket_number(column):
    # Find all sequences of exactly 3 digits
    numeros = re.findall(r'\d{3}', column)

    # Return the first match or None if no match
    return numeros[0] if numeros else None

In [244]:
df_Projects['Nº Ticket Asist'] = df_Projects['Summary'].apply(ticket_number)
df_Projects['Nº Ticket Asist'] = df_Projects['Nº Ticket Asist'].astype(str)

#Exploration and wrangling of Worklogs Database

In [245]:
worklogs

,Worklog Id,Issue Key,Issue Summary,Issue Status,Time Spent (seconds),Start Date,Update Date,Author,Project Name,Issue Type,Due Date,Parent Key
0,6f6b006c-1443-45a1-a749-2b435181f17d,PROJ-497,Owner garden unit environmental perhaps teacher.,Closed,2655,2025-01-23,2025-02-09,Mona King,"Morris, Dyer and Pena",Task,2025-02-28,PROJ-155
1,38f1ab2e-8d85-47b1-9cbb-15a483184651,PROJ-568,Condition local across instead.,Closed,9380,2025-01-29,2025-01-31,Steve Walker,Martinez-Russell,Task,2025-01-30,PROJ-222
2,a34204ba-268b-4961-ae7f-e76987d97ee1,PROJ-731,Site brother nearly yourself bring staff.,Closed,9574,2025-01-18,2025-03-12,Olivia Scott,George-Smith,Epic,2025-01-29,PROJ-577
3,2b7d2e52-fba4-40c8-8f7e-6bb65d702e18,PROJ-615,Home enough amount drop situation section acti...,In Progress,4035,2025-01-31,2025-02-13,Olivia Scott,"Lozano, Parsons and Johnson",Epic,2025-01-12,PROJ-081
4,f8eec32f-3fa7-4a07-9d99-1c52e1978691,PROJ-052,Partner so issue real present necessary although.,Open,6479,2025-02-12,2025-03-12,Olivia Scott,Hardy-Hatfield,Story,2025-03-10,PROJ-015
...,...,...,...,...,...,...,...,...,...,...,...,...
195,89e1c70b-66b6-4d2c-96f2-00aa20760e45,PROJ-600,Southern enough matter support goal account ph...,Open,2212,2025-01-14,2025-03-16,Grace Moore,"Clarke, Dominguez and Pruitt",Epic,2025-01-21,PROJ-506
196,13d417f0-ca1a-44ed-b568-3877c1e23157,PROJ-648,However write leg new watch eye institution.,Resolved,7839,2025-01-11,2025-02-19,Paul Robinson,Thompson-Perez,Bug,2025-01-18,PROJ-307
197,a41a1cd4-f092-4441-943b-dee16257240d,PROJ-206,First program stuff occur.,In Progress,9970,2025-02-25,2025-01-10,Kathy Jackson,Tucker PLC,Bug,2025-03-17,PROJ-470
198,df798203-6a9c-4b61-aa1d-bc3276e64db7,PROJ-112,Writer may success example few land to.,In Progress,8931,2025-02-24,2025-02-11,Paul Robinson,Pittman-Johnson,Bug,2025-01-10,PROJ-786


In [246]:
worklogs.columns

Index(['Worklog Id', 'Issue Key', 'Issue Summary', 'Issue Status',
       'Time Spent (seconds)', 'Start Date', 'Update Date', 'Author',
       'Project Name', 'Issue Type', 'Due Date', 'Parent Key'],
      dtype='object')

In [247]:
columns = ['Worklog Id',
           'Issue Key',
           'Issue Summary',
           'Issue Status',
           'Time Spent (seconds)',
           'Start Date',
           'Update Date',
           'Author',
           'Project Name',
            'Issue Type',
           'Due Date',
            'Parent Key']

In [248]:
worklogs = worklogs[columns]

In [249]:
worklogs['Time Spent (Jours)'] = (worklogs['Time Spent (seconds)']/28800).round(2)
worklogs = worklogs.drop(columns=['Time Spent (seconds)'])

In [250]:
worklogs['Start Date'] = pd.to_datetime(worklogs['Start Date']).dt.tz_localize(None)
worklogs['Week Number'] = worklogs['Start Date'].dt.isocalendar().week
worklogs['Year'] = worklogs['Start Date'].dt.isocalendar().year

In [251]:
project_devs=[
    {'name': 'Alice Johnson', 'role': 'Developer'},
    {'name': 'Bob Smith', 'role': 'Developer'},
    {'name': 'Charlie Brown', 'role': 'Developer'},
    {'name': 'David Williams', 'role': 'Developer'},
    {'name': 'Eva Davis', 'role': 'Developer'},
    {'name': 'Frank Miller', 'role': 'Developer'},
    {'name': 'Grace Moore', 'role': 'Developer'},
    {'name': 'Hannah Taylor', 'role': 'Developer'},
    {'name': 'Ian Anderson', 'role': 'Developer'},
    {'name': 'Jack Thomas', 'role': 'Developer'},
    {'name': 'Kathy Jackson', 'role': 'Developer'},
    {'name': 'Liam White', 'role': 'Developer'},
    {'name': 'Mona King', 'role': 'Developer'},
    {'name': 'Nathan Harris', 'role': 'Developer'},
    {'name': 'Olivia Scott', 'role': 'Developer'},
    {'name': 'Paul Robinson', 'role': 'Tester'},
    {'name': 'Quinn Clark', 'role': 'Tester'},
    {'name': 'Rachel Lewis', 'role': 'Tester'},
    {'name': 'Steve Walker', 'role': 'Tester'},
    {'name': 'Tina Hall', 'role': 'Tester'}
]

In [252]:
author_to_role = {dev['name']: dev['role'] for dev in project_devs}

In [253]:
worklogs['Project'] = worklogs['Author'].map(author_to_role)

In [254]:
#Extracting Asist Ticket Number
worklogs['Nº Ticket Asist'] = worklogs['Issue Summary'].apply(ticket_number)
worklogs['Nº Ticket Asist'] = worklogs['Nº Ticket Asist'].astype(str)

In [255]:
worklogs.isna().sum()

,0
Worklog Id,0
Issue Key,0
Issue Summary,0
Issue Status,0
Start Date,0
Update Date,0
Author,0
Project Name,0
Issue Type,0
Due Date,0


In [256]:
worklogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Worklog Id          200 non-null    object        
 1   Issue Key           200 non-null    object        
 2   Issue Summary       200 non-null    object        
 3   Issue Status        200 non-null    object        
 4   Start Date          200 non-null    datetime64[ns]
 5   Update Date         200 non-null    datetime64[ns]
 6   Author              200 non-null    object        
 7   Project Name        200 non-null    object        
 8   Issue Type          200 non-null    object        
 9   Due Date            200 non-null    datetime64[ns]
 10  Parent Key          200 non-null    object        
 11  Time Spent (Jours)  200 non-null    float64       
 12  Week Number         200 non-null    UInt32        
 13  Year                200 non-null    UInt32        

In [257]:
worklogs.columns = worklogs.columns.str.strip()

In [258]:
worklogs['Parent Key'] = worklogs['Parent Key'].fillna('N/A')
 #Due Date can have n/a values because will not be used in

In [259]:
print(worklogs.columns)

Index(['Worklog Id', 'Issue Key', 'Issue Summary', 'Issue Status',
       'Start Date', 'Update Date', 'Author', 'Project Name', 'Issue Type',
       'Due Date', 'Parent Key', 'Time Spent (Jours)', 'Week Number', 'Year',
       'Project', 'Nº Ticket Asist'],
      dtype='object')


In [260]:
worklogs

,Worklog Id,Issue Key,Issue Summary,Issue Status,Start Date,Update Date,Author,Project Name,Issue Type,Due Date,Parent Key,Time Spent (Jours),Week Number,Year,Project,Nº Ticket Asist
0,6f6b006c-1443-45a1-a749-2b435181f17d,PROJ-497,Owner garden unit environmental perhaps teacher.,Closed,2025-01-23,2025-02-09,Mona King,"Morris, Dyer and Pena",Task,2025-02-28,PROJ-155,0.09,4,2025,Developer,None
1,38f1ab2e-8d85-47b1-9cbb-15a483184651,PROJ-568,Condition local across instead.,Closed,2025-01-29,2025-01-31,Steve Walker,Martinez-Russell,Task,2025-01-30,PROJ-222,0.33,5,2025,Tester,None
2,a34204ba-268b-4961-ae7f-e76987d97ee1,PROJ-731,Site brother nearly yourself bring staff.,Closed,2025-01-18,2025-03-12,Olivia Scott,George-Smith,Epic,2025-01-29,PROJ-577,0.33,3,2025,Developer,None
3,2b7d2e52-fba4-40c8-8f7e-6bb65d702e18,PROJ-615,Home enough amount drop situation section acti...,In Progress,2025-01-31,2025-02-13,Olivia Scott,"Lozano, Parsons and Johnson",Epic,2025-01-12,PROJ-081,0.14,5,2025,Developer,None
4,f8eec32f-3fa7-4a07-9d99-1c52e1978691,PROJ-052,Partner so issue real present necessary although.,Open,2025-02-12,2025-03-12,Olivia Scott,Hardy-Hatfield,Story,2025-03-10,PROJ-015,0.22,7,2025,Developer,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,89e1c70b-66b6-4d2c-96f2-00aa20760e45,PROJ-600,Southern enough matter support goal account ph...,Open,2025-01-14,2025-03-16,Grace Moore,"Clarke, Dominguez and Pruitt",Epic,2025-01-21,PROJ-506,0.08,3,2025,Developer,None
196,13d417f0-ca1a-44ed-b568-3877c1e23157,PROJ-648,However write leg new watch eye institution.,Resolved,2025-01-11,2025-02-19,Paul Robinson,Thompson-Perez,Bug,2025-01-18,PROJ-307,0.27,2,2025,Tester,None
197,a41a1cd4-f092-4441-943b-dee16257240d,PROJ-206,First program stuff occur.,In Progress,2025-02-25,2025-01-10,Kathy Jackson,Tucker PLC,Bug,2025-03-17,PROJ-470,0.35,9,2025,Developer,None
198,df798203-6a9c-4b61-aa1d-bc3276e64db7,PROJ-112,Writer may success example few land to.,In Progress,2025-02-24,2025-02-11,Paul Robinson,Pittman-Johnson,Bug,2025-01-10,PROJ-786,0.31,9,2025,Tester,None


#Edit Asist Databases

### Closed Tickets

In [261]:
db_asist_origin

,0,1,2,3,4,5,6,7,8,9,10
0,Type,Priorité,Numéro,Titre,Statut,Domaine,Date ouverture,Date prise en charge,Date résolution,Date cloture,Groupe Affectation
1,Improvement,High,TICKET-192,Wind effect become month stay.,NaN,Maintenance,2025-02-14 00:00:00,2025-03-17 00:00:00,2025-02-17 00:00:00,2025-02-08 00:00:00,Support Team
2,Bug,Low,TICKET-906,Product important commercial south challenge n...,NaN,Maintenance,2025-02-18 00:00:00,2025-03-03 00:00:00,2025-01-21 00:00:00,2025-03-06 00:00:00,Support Team
3,Task,Urgent,TICKET-586,Worker pass culture social letter someone.,NaN,Development,2025-03-13 00:00:00,2025-01-27 00:00:00,2025-02-26 00:00:00,2025-02-15 00:00:00,Dev Team A
4,Bug,Medium,TICKET-144,Must cost become will seven customer decade.,NaN,Testing,2025-01-31 00:00:00,2025-01-26 00:00:00,2025-03-15 00:00:00,2025-01-21 00:00:00,Dev Team A
...,...,...,...,...,...,...,...,...,...,...,...
996,Bug,Low,TICKET-907,Receive from huge tree value senior effort.,NaN,Support,2025-01-18 00:00:00,2025-03-09 00:00:00,2025-02-04 00:00:00,2025-01-18 00:00:00,QA Team
997,Bug,High,TICKET-839,Better others should firm.,NaN,Testing,2025-03-02 00:00:00,2025-01-05 00:00:00,2025-01-13 00:00:00,2025-02-26 00:00:00,QA Team
998,Task,Low,TICKET-112,Thus and drop without brother.,NaN,Testing,2025-02-13 00:00:00,2025-03-17 00:00:00,2025-01-28 00:00:00,2025-01-06 00:00:00,Support Team
999,Bug,Low,TICKET-163,She science write rule.,NaN,Support,2025-02-04 00:00:00,2025-01-30 00:00:00,2025-02-06 00:00:00,2025-01-21 00:00:00,Support Team


In [262]:
#Removing First Lines
db_asist_origin.rename(columns = db_asist_origin.iloc[0], inplace=True)
db_asist_origin = db_asist_origin.drop([0], axis=0)
db_asist_origin.reset_index(drop=True, inplace=True)

In [263]:
db_asist = db_asist_origin

In [264]:
db_asist

,Type,Priorité,Numéro,Titre,Statut,Domaine,Date ouverture,Date prise en charge,Date résolution,Date cloture,Groupe Affectation
0,Improvement,High,TICKET-192,Wind effect become month stay.,NaN,Maintenance,2025-02-14 00:00:00,2025-03-17 00:00:00,2025-02-17 00:00:00,2025-02-08 00:00:00,Support Team
1,Bug,Low,TICKET-906,Product important commercial south challenge n...,NaN,Maintenance,2025-02-18 00:00:00,2025-03-03 00:00:00,2025-01-21 00:00:00,2025-03-06 00:00:00,Support Team
2,Task,Urgent,TICKET-586,Worker pass culture social letter someone.,NaN,Development,2025-03-13 00:00:00,2025-01-27 00:00:00,2025-02-26 00:00:00,2025-02-15 00:00:00,Dev Team A
3,Bug,Medium,TICKET-144,Must cost become will seven customer decade.,NaN,Testing,2025-01-31 00:00:00,2025-01-26 00:00:00,2025-03-15 00:00:00,2025-01-21 00:00:00,Dev Team A
4,Feature Request,Medium,TICKET-149,System project energy series shake amount disc...,NaN,Testing,2025-01-10 00:00:00,2025-02-07 00:00:00,2025-02-03 00:00:00,2025-03-16 00:00:00,Support Team
...,...,...,...,...,...,...,...,...,...,...,...
995,Bug,Low,TICKET-907,Receive from huge tree value senior effort.,NaN,Support,2025-01-18 00:00:00,2025-03-09 00:00:00,2025-02-04 00:00:00,2025-01-18 00:00:00,QA Team
996,Bug,High,TICKET-839,Better others should firm.,NaN,Testing,2025-03-02 00:00:00,2025-01-05 00:00:00,2025-01-13 00:00:00,2025-02-26 00:00:00,QA Team
997,Task,Low,TICKET-112,Thus and drop without brother.,NaN,Testing,2025-02-13 00:00:00,2025-03-17 00:00:00,2025-01-28 00:00:00,2025-01-06 00:00:00,Support Team
998,Bug,Low,TICKET-163,She science write rule.,NaN,Support,2025-02-04 00:00:00,2025-01-30 00:00:00,2025-02-06 00:00:00,2025-01-21 00:00:00,Support Team


In [265]:
db_asist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Type                  1000 non-null   object
 1   Priorité              1000 non-null   object
 2   Numéro                1000 non-null   object
 3   Titre                 1000 non-null   object
 4   Statut                0 non-null      object
 5   Domaine               1000 non-null   object
 6   Date ouverture        1000 non-null   object
 7   Date prise en charge  1000 non-null   object
 8   Date résolution       1000 non-null   object
 9   Date cloture          1000 non-null   object
 10  Groupe Affectation    1000 non-null   object
dtypes: object(11)
memory usage: 86.1+ KB


In [266]:
#Adding a column with the Status , due to the fact that this table has all the closed tickets
db_asist.loc[:,'Statut'] = 'Résolu'
db_asist.loc[:,'Priorité'] = np.random.choice(['Normale', 'Haute', 'Bas'], size=1000)

In [267]:
#db_asist.loc[:,'Date ouverture'] = pd.to_datetime(db_asist['Date ouverture'].str[:10], format='%d/%m/%Y')
#db_asist.loc[:,'Date prise en charge'] = pd.to_datetime(db_asist['Date prise en charge'].str[:10], format='%d/%m/%Y')
#db_asist.loc[:,'Date résolution'] = pd.to_datetime(db_asist['Date résolution'].str[:10], format='%d/%m/%Y')
#db_asist.loc[:,'Date cloture'] = pd.to_datetime(db_asist['Date cloture'].str[:10], format='%d/%m/%Y')

In [268]:
db_asist

,Type,Priorité,Numéro,Titre,Statut,Domaine,Date ouverture,Date prise en charge,Date résolution,Date cloture,Groupe Affectation
0,Improvement,Bas,TICKET-192,Wind effect become month stay.,Résolu,Maintenance,2025-02-14 00:00:00,2025-03-17 00:00:00,2025-02-17 00:00:00,2025-02-08 00:00:00,Support Team
1,Bug,Bas,TICKET-906,Product important commercial south challenge n...,Résolu,Maintenance,2025-02-18 00:00:00,2025-03-03 00:00:00,2025-01-21 00:00:00,2025-03-06 00:00:00,Support Team
2,Task,Normale,TICKET-586,Worker pass culture social letter someone.,Résolu,Development,2025-03-13 00:00:00,2025-01-27 00:00:00,2025-02-26 00:00:00,2025-02-15 00:00:00,Dev Team A
3,Bug,Haute,TICKET-144,Must cost become will seven customer decade.,Résolu,Testing,2025-01-31 00:00:00,2025-01-26 00:00:00,2025-03-15 00:00:00,2025-01-21 00:00:00,Dev Team A
4,Feature Request,Normale,TICKET-149,System project energy series shake amount disc...,Résolu,Testing,2025-01-10 00:00:00,2025-02-07 00:00:00,2025-02-03 00:00:00,2025-03-16 00:00:00,Support Team
...,...,...,...,...,...,...,...,...,...,...,...
995,Bug,Haute,TICKET-907,Receive from huge tree value senior effort.,Résolu,Support,2025-01-18 00:00:00,2025-03-09 00:00:00,2025-02-04 00:00:00,2025-01-18 00:00:00,QA Team
996,Bug,Bas,TICKET-839,Better others should firm.,Résolu,Testing,2025-03-02 00:00:00,2025-01-05 00:00:00,2025-01-13 00:00:00,2025-02-26 00:00:00,QA Team
997,Task,Normale,TICKET-112,Thus and drop without brother.,Résolu,Testing,2025-02-13 00:00:00,2025-03-17 00:00:00,2025-01-28 00:00:00,2025-01-06 00:00:00,Support Team
998,Bug,Normale,TICKET-163,She science write rule.,Résolu,Support,2025-02-04 00:00:00,2025-01-30 00:00:00,2025-02-06 00:00:00,2025-01-21 00:00:00,Support Team


### Asist Open Tickets

In [269]:
db_asist_open.rename(columns = db_asist_open.iloc[0], inplace=True)
db_asist_open = db_asist_open.drop([0], axis=0)
db_asist_open.reset_index(drop=True, inplace=True)

In [270]:
db_asist_open.rename(columns={'Date/heure prise en charge (vide si non renseignée)': 'Date prise en charge',
                     'Date/heure résolution (vide si non renseignée)': 'Date résolution', 'Date/heure cloture (vide si non renseignée)':'Date cloture',
                             'Numéro ticket':'Numéro', "Groupe(s) d'attribution":'Groupe Affectation'}, inplace = True)

In [271]:
db_asist_open

,Type,Priorité,Numéro,Titre,Statut,Domaine,Date ouverture,Date prise en charge,Date résolution,Date cloture,Groupe Affectation
0,Improvement,Medium,TICKET-648,Here federal arm position.,Open,Maintenance,2025-01-12 00:00:00,2025-03-05 00:00:00,NaN,NaN,Support Team
1,Bug,Medium,TICKET-866,Unit reduce trip knowledge action certain stuff.,Resolved,Support,2025-01-10 00:00:00,2025-01-03 00:00:00,NaN,NaN,Dev Team A
2,Improvement,Low,TICKET-076,Learn deal discuss front bar.,Open,Testing,2025-01-02 00:00:00,2025-02-24 00:00:00,NaN,NaN,Support Team
3,Task,Medium,TICKET-287,Exactly sort body industry these represent.,Resolved,Testing,2025-02-18 00:00:00,2025-01-11 00:00:00,NaN,NaN,Dev Team B
4,Bug,Urgent,TICKET-083,Front wish adult discussion.,Closed,Support,2025-03-17 00:00:00,2025-01-02 00:00:00,NaN,NaN,Support Team
...,...,...,...,...,...,...,...,...,...,...,...
495,Feature Request,Medium,TICKET-995,Mind woman mother quality.,Resolved,Maintenance,2025-01-05 00:00:00,2025-02-02 00:00:00,NaN,NaN,QA Team
496,Feature Request,Low,TICKET-273,Too poor catch color much.,In Progress,Development,2025-02-28 00:00:00,2025-02-21 00:00:00,NaN,NaN,Dev Team A
497,Feature Request,Medium,TICKET-498,Recognize mission none face at story glass.,Open,Support,2025-02-26 00:00:00,2025-02-15 00:00:00,NaN,NaN,Dev Team A
498,Bug,Medium,TICKET-049,Key pass doctor standard.,Resolved,Testing,2025-01-18 00:00:00,2025-02-28 00:00:00,NaN,NaN,QA Team


### Merge Databases

In [272]:
Columns = ['Type', 'Priorité', 'Numéro', 'Titre', 'Statut', 'Domaine', 'Date ouverture', 'Date prise en charge', 'Date résolution',
'Date cloture', 'Groupe Affectation']

In [273]:
merged_db_asist = pd.concat([db_asist_open[Columns],db_asist[Columns]],ignore_index=True, sort=False)

In [274]:
merged_db_asist

,Type,Priorité,Numéro,Titre,Statut,Domaine,Date ouverture,Date prise en charge,Date résolution,Date cloture,Groupe Affectation
0,Improvement,Medium,TICKET-648,Here federal arm position.,Open,Maintenance,2025-01-12 00:00:00,2025-03-05 00:00:00,NaN,NaN,Support Team
1,Bug,Medium,TICKET-866,Unit reduce trip knowledge action certain stuff.,Resolved,Support,2025-01-10 00:00:00,2025-01-03 00:00:00,NaN,NaN,Dev Team A
2,Improvement,Low,TICKET-076,Learn deal discuss front bar.,Open,Testing,2025-01-02 00:00:00,2025-02-24 00:00:00,NaN,NaN,Support Team
3,Task,Medium,TICKET-287,Exactly sort body industry these represent.,Resolved,Testing,2025-02-18 00:00:00,2025-01-11 00:00:00,NaN,NaN,Dev Team B
4,Bug,Urgent,TICKET-083,Front wish adult discussion.,Closed,Support,2025-03-17 00:00:00,2025-01-02 00:00:00,NaN,NaN,Support Team
...,...,...,...,...,...,...,...,...,...,...,...
1495,Bug,Haute,TICKET-907,Receive from huge tree value senior effort.,Résolu,Support,2025-01-18 00:00:00,2025-03-09 00:00:00,2025-02-04 00:00:00,2025-01-18 00:00:00,QA Team
1496,Bug,Bas,TICKET-839,Better others should firm.,Résolu,Testing,2025-03-02 00:00:00,2025-01-05 00:00:00,2025-01-13 00:00:00,2025-02-26 00:00:00,QA Team
1497,Task,Normale,TICKET-112,Thus and drop without brother.,Résolu,Testing,2025-02-13 00:00:00,2025-03-17 00:00:00,2025-01-28 00:00:00,2025-01-06 00:00:00,Support Team
1498,Bug,Normale,TICKET-163,She science write rule.,Résolu,Support,2025-02-04 00:00:00,2025-01-30 00:00:00,2025-02-06 00:00:00,2025-01-21 00:00:00,Support Team


In [275]:
merged_db_asist.isna().sum()

,0
Type,0
Priorité,0
Numéro,0
Titre,0
Statut,0
Domaine,0
Date ouverture,0
Date prise en charge,0
Date résolution,500
Date cloture,500


In [276]:
merged_db_asist['Domaine'] = merged_db_asist['Domaine'].fillna('Autres')


#Adding the estimative and Time Spent to Asist Database

In [277]:
df_Projects['Nº Ticket Asist'] = 'TICKET-' + df_Projects['Nº Ticket Asist'].astype(str)

In [278]:
merged_db_asist['Numéro'] = merged_db_asist['Numéro'].astype(str)

# Agrupar os dados por 'Nº Ticket Asist' e somar as colunas relevantes
df_Projects_grouped = df_Projects.groupby('Nº Ticket Asist', as_index=False)[
    ['Σ Original Estimate', 'Σ Time Spent']
].sum()

# Realizar o merge entre as tabelas para adicionar as colunas 'Σ Remaining Estimate' e 'Σ Time Spent'
merged_db_asist = pd.merge(
    merged_db_asist,
    df_Projects_grouped[['Nº Ticket Asist', 'Σ Original Estimate', 'Σ Time Spent']],
    left_on='Numéro',
    right_on='Nº Ticket Asist',
    how='left'
)

# Substituir valores NaN por 0 nas colunas 'Σ Original Estimate', 'Σ Remaining Estimate' e 'Σ Time Spent'
merged_db_asist[['Σ Original Estimate', 'Σ Time Spent']] = merged_db_asist[['Σ Original Estimate', 'Σ Time Spent']].fillna(0)


In [279]:
merged_db_asist['Projeto_TMA'] = merged_db_asist.apply(
    lambda row: 'TMA' if row['Σ Original Estimate'] < 5 else 'Projet', axis=1
)

In [280]:
merged_db_asist = merged_db_asist.drop_duplicates(subset=['Numéro'])
merged_db_asist= merged_db_asist[merged_db_asist['Groupe Affectation'].str.contains('TMA', na=False)]
merged_db_asist.reset_index(drop=True, inplace=True)

In [281]:
merged_db_asist

,Type,Priorité,Numéro,Titre,Statut,Domaine,Date ouverture,Date prise en charge,Date résolution,Date cloture,Groupe Affectation,Nº Ticket Asist,Σ Original Estimate,Σ Time Spent,Projeto_TMA


#Adding TMA/PROJECT to worklogs

In [282]:
merged_worklogs = worklogs


In [283]:
worklogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Worklog Id          200 non-null    object        
 1   Issue Key           200 non-null    object        
 2   Issue Summary       200 non-null    object        
 3   Issue Status        200 non-null    object        
 4   Start Date          200 non-null    datetime64[ns]
 5   Update Date         200 non-null    datetime64[ns]
 6   Author              200 non-null    object        
 7   Project Name        200 non-null    object        
 8   Issue Type          200 non-null    object        
 9   Due Date            200 non-null    datetime64[ns]
 10  Parent Key          200 non-null    object        
 11  Time Spent (Jours)  200 non-null    float64       
 12  Week Number         200 non-null    UInt32        
 13  Year                200 non-null    UInt32        

In [284]:
print(merged_db_asist['Numéro'].duplicated().sum())
merged_db_asist = merged_db_asist.drop_duplicates(subset=['Numéro'])


0


In [285]:
merged_worklogs['Nº Ticket Asist'] = merged_worklogs['Nº Ticket Asist'].astype(str)
merged_db_asist['Numéro'] = merged_db_asist['Numéro'].astype(str)

merged_worklogs = pd.merge(
    merged_worklogs,
    merged_db_asist[['Projeto_TMA','Numéro', 'Domaine', 'Type', 'Σ Original Estimate']],
    left_on='Nº Ticket Asist',
    right_on='Numéro',
    how='left'
)

merged_worklogs.fillna({'Projeto_TMA': 'Unknown', 'Domaine': 'Autres', 'Type': 'Undefined'}, inplace=True)


In [286]:
#merged_worklogs = pd.concat([worklogs_edited, merged_worklogs])

In [287]:
merged_worklogs = merged_worklogs.drop_duplicates(subset=['Worklog Id'], keep='last')


In [288]:
merged_worklogs

,Worklog Id,Issue Key,Issue Summary,Issue Status,Start Date,Update Date,Author,Project Name,Issue Type,Due Date,...,Time Spent (Jours),Week Number,Year,Project,Nº Ticket Asist,Projeto_TMA,Numéro,Domaine,Type,Σ Original Estimate
0,6f6b006c-1443-45a1-a749-2b435181f17d,PROJ-497,Owner garden unit environmental perhaps teacher.,Closed,2025-01-23,2025-02-09,Mona King,"Morris, Dyer and Pena",Task,2025-02-28,...,0.09,4,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN
1,38f1ab2e-8d85-47b1-9cbb-15a483184651,PROJ-568,Condition local across instead.,Closed,2025-01-29,2025-01-31,Steve Walker,Martinez-Russell,Task,2025-01-30,...,0.33,5,2025,Tester,None,Unknown,NaN,Autres,Undefined,NaN
2,a34204ba-268b-4961-ae7f-e76987d97ee1,PROJ-731,Site brother nearly yourself bring staff.,Closed,2025-01-18,2025-03-12,Olivia Scott,George-Smith,Epic,2025-01-29,...,0.33,3,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN
3,2b7d2e52-fba4-40c8-8f7e-6bb65d702e18,PROJ-615,Home enough amount drop situation section acti...,In Progress,2025-01-31,2025-02-13,Olivia Scott,"Lozano, Parsons and Johnson",Epic,2025-01-12,...,0.14,5,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN
4,f8eec32f-3fa7-4a07-9d99-1c52e1978691,PROJ-052,Partner so issue real present necessary although.,Open,2025-02-12,2025-03-12,Olivia Scott,Hardy-Hatfield,Story,2025-03-10,...,0.22,7,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,89e1c70b-66b6-4d2c-96f2-00aa20760e45,PROJ-600,Southern enough matter support goal account ph...,Open,2025-01-14,2025-03-16,Grace Moore,"Clarke, Dominguez and Pruitt",Epic,2025-01-21,...,0.08,3,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN
196,13d417f0-ca1a-44ed-b568-3877c1e23157,PROJ-648,However write leg new watch eye institution.,Resolved,2025-01-11,2025-02-19,Paul Robinson,Thompson-Perez,Bug,2025-01-18,...,0.27,2,2025,Tester,None,Unknown,NaN,Autres,Undefined,NaN
197,a41a1cd4-f092-4441-943b-dee16257240d,PROJ-206,First program stuff occur.,In Progress,2025-02-25,2025-01-10,Kathy Jackson,Tucker PLC,Bug,2025-03-17,...,0.35,9,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN
198,df798203-6a9c-4b61-aa1d-bc3276e64db7,PROJ-112,Writer may success example few land to.,In Progress,2025-02-24,2025-02-11,Paul Robinson,Pittman-Johnson,Bug,2025-01-10,...,0.31,9,2025,Tester,None,Unknown,NaN,Autres,Undefined,NaN


In [289]:
#Adding the classification for each dev/tester


merged_worklogs['Dev/Tester'] = merged_worklogs['Author'].apply(lambda x: author_to_role.get(x, 'Unknown'))
merged_worklogs['HN/STACI'] = merged_worklogs['Author'].apply(lambda x: 'HN' if x in author_to_role else 'CLIENT')


In [290]:
merged_worklogs

,Worklog Id,Issue Key,Issue Summary,Issue Status,Start Date,Update Date,Author,Project Name,Issue Type,Due Date,...,Year,Project,Nº Ticket Asist,Projeto_TMA,Numéro,Domaine,Type,Σ Original Estimate,Dev/Tester,HN/STACI
0,6f6b006c-1443-45a1-a749-2b435181f17d,PROJ-497,Owner garden unit environmental perhaps teacher.,Closed,2025-01-23,2025-02-09,Mona King,"Morris, Dyer and Pena",Task,2025-02-28,...,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN,Developer,HN
1,38f1ab2e-8d85-47b1-9cbb-15a483184651,PROJ-568,Condition local across instead.,Closed,2025-01-29,2025-01-31,Steve Walker,Martinez-Russell,Task,2025-01-30,...,2025,Tester,None,Unknown,NaN,Autres,Undefined,NaN,Tester,HN
2,a34204ba-268b-4961-ae7f-e76987d97ee1,PROJ-731,Site brother nearly yourself bring staff.,Closed,2025-01-18,2025-03-12,Olivia Scott,George-Smith,Epic,2025-01-29,...,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN,Developer,HN
3,2b7d2e52-fba4-40c8-8f7e-6bb65d702e18,PROJ-615,Home enough amount drop situation section acti...,In Progress,2025-01-31,2025-02-13,Olivia Scott,"Lozano, Parsons and Johnson",Epic,2025-01-12,...,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN,Developer,HN
4,f8eec32f-3fa7-4a07-9d99-1c52e1978691,PROJ-052,Partner so issue real present necessary although.,Open,2025-02-12,2025-03-12,Olivia Scott,Hardy-Hatfield,Story,2025-03-10,...,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN,Developer,HN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,89e1c70b-66b6-4d2c-96f2-00aa20760e45,PROJ-600,Southern enough matter support goal account ph...,Open,2025-01-14,2025-03-16,Grace Moore,"Clarke, Dominguez and Pruitt",Epic,2025-01-21,...,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN,Developer,HN
196,13d417f0-ca1a-44ed-b568-3877c1e23157,PROJ-648,However write leg new watch eye institution.,Resolved,2025-01-11,2025-02-19,Paul Robinson,Thompson-Perez,Bug,2025-01-18,...,2025,Tester,None,Unknown,NaN,Autres,Undefined,NaN,Tester,HN
197,a41a1cd4-f092-4441-943b-dee16257240d,PROJ-206,First program stuff occur.,In Progress,2025-02-25,2025-01-10,Kathy Jackson,Tucker PLC,Bug,2025-03-17,...,2025,Developer,None,Unknown,NaN,Autres,Undefined,NaN,Developer,HN
198,df798203-6a9c-4b61-aa1d-bc3276e64db7,PROJ-112,Writer may success example few land to.,In Progress,2025-02-24,2025-02-11,Paul Robinson,Pittman-Johnson,Bug,2025-01-10,...,2025,Tester,None,Unknown,NaN,Autres,Undefined,NaN,Tester,HN


# Extracting Insights for Report

In [291]:
Start_Date = '2025-01-01'
End_Date = '2025-03-12'

In [292]:
merged_worklogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Worklog Id           200 non-null    object        
 1   Issue Key            200 non-null    object        
 2   Issue Summary        200 non-null    object        
 3   Issue Status         200 non-null    object        
 4   Start Date           200 non-null    datetime64[ns]
 5   Update Date          200 non-null    datetime64[ns]
 6   Author               200 non-null    object        
 7   Project Name         200 non-null    object        
 8   Issue Type           200 non-null    object        
 9   Due Date             200 non-null    datetime64[ns]
 10  Parent Key           200 non-null    object        
 11  Time Spent (Jours)   200 non-null    float64       
 12  Week Number          200 non-null    UInt32        
 13  Year                 200 non-null  

In [293]:
merged_worklogs['Start Date'] = pd.to_datetime(merged_worklogs['Start Date'], errors='coerce')

merged_worklogs_filtered = merged_worklogs[
    (merged_worklogs['Start Date'] >= Start_Date) &
    (merged_worklogs['Start Date'] <= End_Date)
]

### Pivot Table: Time Spent during the month per Type (Project or TMA)

In [294]:
merged_worklogs_filtered = merged_worklogs_filtered[merged_worklogs_filtered['Project'] != 'ECATS']

In [295]:
# Create pivot table
pivot_table = pd.pivot_table(
    merged_worklogs_filtered,
    index='Projeto_TMA',
    values='Time Spent (Jours)',
    columns='Domaine',
    aggfunc='sum',
    fill_value=0  # Ensures no NaN values
)

# Convert values to numeric (in case of unexpected types)
pivot_table = pivot_table.apply(pd.to_numeric, errors='coerce')

# Add total column (sum of all values across columns for each row)
pivot_table["Total"] = pivot_table.sum(axis=1)

# Add total row (sum of all values across rows for each column)
pivot_table.loc["Total"] = pivot_table.sum(axis=0)

In [296]:
pivot_table

Domaine,Autres,Total
Projeto_TMA,,
Unknown,36.39,36.39
Total,36.39,36.39


### Time Spent on 'Non Facturable' actions

In [297]:

df_nonfact = merged_worklogs_filtered[merged_worklogs_filtered['Project Name'] == 'George-Smith']

df_grouped = df_nonfact.groupby(['Project']).agg({
    'Time Spent (Jours)': 'sum'
}).reset_index()

df_grouped.loc['Total'] = df_grouped.sum(axis=0)

df_grouped



,Project,Time Spent (Jours)
0,Developer,0.33
Total,Developer,0.33


### Time Spent per Dev/Tester per week

In [298]:
Time_per_week = merged_worklogs_filtered.groupby(['Author', 'Project', 'Year', 'Week Number']).agg({'Time Spent (Jours)': 'sum'})

Time_per_week['Status'] = Time_per_week['Time Spent (Jours)'].apply(lambda x: 'OK' if x == 5 else 'NOK')

Time_per_week

Time Spent (Jours) Status
Author        Project   Year Week Number                           
Alice Johnson Developer 2025 3                          0.22    NOK
                             5                          0.26    NOK
                             8                          0.42    NOK
                             9                          0.53    NOK
                             11                         0.30    NOK
...                                                      ...    ...
Tina Hall     Tester    2025 4                          0.30    NOK
                             5                          0.26    NOK
                             7                          0.12    NOK
                             8                          0.24    NOK
                             9                          0.13    NOK

[124 rows x 2 columns]

## Projects timings

Show per project: Time Spent on Development, Original estimate of project, Time of development spent on Bugs opened by Staci, time of development spent on bugs opened by HN, Time spent on Testes vs original estimate, Number of bugs opened by HN and Staci, AVG solution time

 ### Time Spent on Initial development of project (Excluding Bugs)

In [300]:
dev_filtered = df_Projects[
    ((df_Projects['Issue Type'] != 'Bug') |
    (~df_Projects['Summary'].str.contains('BUG', case=False, na=False))) &
    (df_Projects['Project name'].str.contains('RECATS', case=False, na=False))
]
Dev_time = dev_filtered.groupby(['Project name']).agg({'Σ Time Spent': 'sum', 'Σ Original Estimate': 'sum'})
Dev_time.loc['Total'] = Dev_time.sum(axis=0)
Dev_time

,Σ Time Spent,Σ Original Estimate
Project name,,
Total,0.0,0.0


### Time Spent on Solving Bugs (BUG + Test)




In [301]:
df_Projects

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Parent,Project name,Assignee,Creator,Created,Updated,Σ Original Estimate,Σ Time Spent,Nº Ticket Asist
0,TICKET-735 Help record his.,ISSUE-308,7228,Feature,Resolved,PROJECT-833,N/A,"Rodriguez, Nash and Vargas",Bryan Cobb,Aaron Levy,2025-02-14,2025-02-06,0.0,0.0,TICKET-735
1,TICKET-761 Though suggest Mrs indeed pressure ...,ISSUE-911,5822,Improvement,Resolved,PROJECT-179,N/A,"Odom, Phillips and Vang",Adam Ho,Catherine Monroe,2025-01-23,2025-01-25,0.0,0.0,TICKET-761
2,TICKET-889 Various color and quite investment ...,ISSUE-318,7829,Bug,Closed,PROJECT-000,ISSUE-491,Miller LLC,Shelley Pollard,Kelsey Harris,2025-02-06,2025-02-11,0.0,0.0,TICKET-889
3,TICKET-442 Charge law new simple science parti...,ISSUE-554,3230,Bug,Open,PROJECT-855,ISSUE-866,Acosta PLC,Sheryl Williams,Charlene Watson,2025-02-11,2025-02-09,0.0,0.0,TICKET-442
4,TICKET-764 Necessary brother believe boy.,ISSUE-149,8526,Improvement,Closed,PROJECT-060,ISSUE-962,White-Patterson,John Pitts,Beth Hull,2025-01-24,2025-01-30,0.0,0.0,TICKET-764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,TICKET-951 Professional charge commercial itself.,ISSUE-497,7713,Improvement,Closed,PROJECT-655,ISSUE-136,Newman-Sandoval,Kristin Ramirez,William Randolph,2025-03-17,2025-02-07,0.0,0.0,TICKET-951
196,TICKET-852 Evidence true quality.,ISSUE-888,4250,Bug,Closed,PROJECT-066,N/A,"Baldwin, Barnett and Koch",Glen Ortiz,Elizabeth Harris,2025-01-24,2025-01-05,0.0,0.0,TICKET-852
197,TICKET-270 If likely base sound how although s...,ISSUE-558,1689,Bug,Resolved,PROJECT-639,N/A,"Calderon, Clark and Fox",Shaun Myers,Ronald Hoffman,2025-01-12,2025-01-14,0.0,0.0,TICKET-270
198,TICKET-119 General listen democratic tonight f...,ISSUE-699,9382,Improvement,Open,PROJECT-552,N/A,Martinez Inc,Michele Zimmerman,Samuel Baxter,2025-01-25,2025-03-17,0.0,0.0,TICKET-119


In [302]:
bugs_filtered = df_Projects[
    ((df_Projects['Issue Type'] == 'Bug') |
    (df_Projects['Summary'].str.contains('BUG', case=False, na=False)))
]
bug_time = bugs_filtered.groupby(['Project name']).agg({'Σ Time Spent': 'sum'})
bug_time.loc['Total'] = bug_time.sum(axis=0)
bug_time

,Σ Time Spent
Project name,
Acosta PLC,0.0
"Anthony, Chambers and Price",0.0
"Baldwin, Barnett and Koch",0.0
"Calderon, Clark and Fox",0.0
Carlson LLC,0.0
"Chavez, Harris and Morrison",0.0
Costa-Alexander,0.0
Cox LLC,0.0
"Crosby, Watson and Jones",0.0


### Time Spent on Writing and executing Tests


In [303]:
Tests_filtered = df_Projects[
    ((df_Projects['Issue Type'] == 'Recette/Testing') |
    (df_Projects['Summary'].str.contains('Testing', case=False, na=False))) &
    (df_Projects['Project name'].str.contains('RECATS', case=False, na=False))
]
Tests = Tests_filtered.groupby(['Project name']).agg({'Σ Time Spent': 'sum',  'Σ Original Estimate': 'sum'})
Tests.loc['Total'] = Tests.sum(axis=0)
Tests

,Σ Time Spent,Σ Original Estimate
Project name,,
Total,0.0,0.0


### Number of Bugs Opened By Client vs HN


In [304]:
df_Projects.loc[:, 'HN/STACI'] = df_Projects['Creator'].apply(lambda x: 'HN' if x in all else 'STACI')

In [305]:
Bugs = df_Projects[
    ((df_Projects['Issue Type'] != 'Bug') |
    (~df_Projects['Summary'].str.contains('BUG', case=False, na=False))) &
    (df_Projects['Project name'].str.contains('RECATS', case=False, na=False))
]
Bugs_count = Bugs.groupby(['Project name', 'HN/STACI']).agg({'Σ Time Spent': 'count'}).reset_index()
Bugs_count_unstacked = Bugs_count.set_index(['Project name', 'HN/STACI'])['Σ Time Spent'].unstack(fill_value=0).reset_index()
Bugs_count.loc['Total'] = Bugs_count.sum(axis=0)
Bugs_count

,Project name,HN/STACI,Σ Time Spent
Total,0,0,0


### AVG Solution Time

In [306]:
Solution_time = df_Projects[
    ((df_Projects['Issue Type'] != 'Bug') |
    (~df_Projects['Summary'].str.contains('BUG', case=False, na=False))) &
    (df_Projects['Project name'].str.contains('RECATS', case=False, na=False))
]
Solution_time = Solution_time.groupby(['Project name', 'HN/STACI']).agg({'Σ Time Spent': 'mean'}).reset_index().round(2)
Solution_time = Solution_time.set_index(['Project name', 'HN/STACI'])['Σ Time Spent'].unstack(fill_value=0).reset_index()
Solution_time.loc['Total'] = Solution_time.sum(axis=0)
Solution_time

HN/STACI,Project name
Total,0


##Machine Learning Methods

Will separate the dataset 'db_asist' in two. The tickets classified as closed will be used during training phase

### Forecast of Time Spent on ticket facturation for project billing optimization - with Random Forest Regressor

In [ ]:
df = merged_db_asist[merged_db_asist['Statut'] == 'Résolu']

In [ ]:
df = df[df['Σ Time Spent'] != 0]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140 entries, 143 to 2859
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Type                  140 non-null    object 
 1   Priorité              140 non-null    object 
 2   Numéro                140 non-null    object 
 3   Titre                 140 non-null    object 
 4   Statut                140 non-null    object 
 5   Domaine               140 non-null    object 
 6   Date ouverture        140 non-null    object 
 7   Date prise en charge  140 non-null    object 
 8   Date résolution       140 non-null    object 
 9   Date cloture          140 non-null    object 
 10  Groupe Affectation    140 non-null    object 
 11  Nº Ticket Asist       140 non-null    object 
 12  Σ Original Estimate   140 non-null    float64
 13  Σ Time Spent          140 non-null    float64
 14  Projeto_TMA           140 non-null    object 
dtypes: float64(2), object(13)

In [ ]:
df['Priorité'].unique()

array(['Normale', 'Haute', 'Bas'], dtype=object)

In [ ]:
df.isna().sum()

,0
Type,0
Priorité,0
Numéro,0
Titre,0
Statut,0
Domaine,0
Date ouverture,0
Date prise en charge,0
Date résolution,0
Date cloture,0


In [ ]:
df.loc[:,'Titre'] = df['Titre'].astype(str)

In [ ]:
# Pré-processamento de texto (Descrição dos tickets)
tfidf = TfidfVectorizer(stop_words='english')
X_text = tfidf.fit_transform(df['Titre'])

tfidf_df = pd.DataFrame(X_text.toarray(), columns=tfidf.get_feature_names_out())


In [ ]:
transformer = ColumnTransformer(
    [('vec', TfidfVectorizer(), df['Titre'])],   # column should be a string or int
    remainder='passthrough'
)

In [ ]:
#Creating independent and dependent variables
X = pd.concat([df[['Priorité', 'Type']], pd.DataFrame(X_text.toarray())], axis=1)

y = df['Σ Time Spent']
y = df.rename(columns={"Σ Time Spent": "Time Spent"})["Time Spent"]


In [ ]:
y

,Time Spent
143,0.12
247,1.28
251,0.44
259,3.93
280,0.25
...,...
2598,1.55
2816,0.03
2819,0.38
2832,0.56


In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform="pandas")

le = LabelEncoder()

#X_encoded = enc.fit_transform(df[['Priorité', 'Type']])

le_priorite = LabelEncoder()
df['Priorité_encoded'] = le_priorite.fit_transform(df['Priorité'])

le_type = LabelEncoder()
df['Type_encoded'] = le_type.fit_transform(df['Type'])
X_encoded = df[['Priorité_encoded', 'Type_encoded']]


X = pd.concat([X_encoded, tfidf_df], axis=1)
X


,Priorité_encoded,Type_encoded,0240_,1apispec,1specadi,1specedi,2405150001,2757,3476,3476_urmet,...,webservices,wms,xml,yoplait,écran,édition,éditions,état,évolution,évolutions
143,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.439577,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y

,Time Spent
143,0.12
247,1.28
251,0.44
259,3.93
280,0.25
...,...
2598,1.55
2816,0.03
2819,0.38
2832,0.56


In [ ]:

df = df.dropna(subset=['Priorité'])  # Ajuste 'priority' conforme necessário
X = X.loc[df.index]  # Atualize X para manter as mesmas linhas de df
y = df['Σ Time Spent']  # Atualize y com a coluna 'priority' de df


y = y[X.index]

print(len(X), len(y))

140 140


In [ ]:
#Training the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")


mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 1.0686571428571428
Mean Squared Error (MSE): 3.0700466035714293
R² Score: -0.013847154398396011


The values found for the model sugests that the model is not performing well. One of the hipothesis we don't have enough data to tests and the model would need to be applied when we gather enough data.

Another solution is to analyse which variable has a higher correlation to the time spent so we could gather some insights

In [ ]:
label_encoder = LabelEncoder()

categorical_columns = df.select_dtypes(include=['object']).columns


for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

In [ ]:

corr_matrix = df.corr()

time_spent_corr = corr_matrix["Σ Time Spent"]

time_spent_corr = time_spent_corr.drop("Σ Time Spent").sort_values(ascending=False)

print(time_spent_corr)

Σ Original Estimate     0.825785
Date cloture            0.170782
Groupe Affectation      0.162588
Date résolution         0.162560
Date prise en charge    0.139463
Type_encoded            0.073484
Type                    0.073484
Titre                   0.004734
Date ouverture         -0.009996
Numéro                 -0.016241
Nº Ticket Asist        -0.016241
Domaine                -0.269807
Priorité_encoded       -0.276712
Priorité               -0.276712
Projeto_TMA            -0.677919
Statut                       NaN
Name: Σ Time Spent, dtype: float64


As we can see only original estimate has a strong correlation to our taget variable, as we can see the variables that the management wanted to use (Priorité and Type) have a really low correlation indication that it cannot be used for a prediction of Time Spent

In [ ]:
#Creating independent and dependent variables
X = df['Σ Original Estimate']

y = df['Σ Time Spent']
y = df.rename(columns={"Σ Time Spent": "Time Spent"})["Time Spent"]


In [ ]:
# Se você tiver uma única coluna em X_train, converta para 2D
X = X.values.reshape(-1, 1)

In [ ]:
#Training the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")


mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 1.0686571428571428
Mean Squared Error (MSE): 3.0700466035714293
R² Score: -0.013847154398396011


The values are the same even with a higher correlated variable, we will test now to apply Linear Regression to test how it will perform

In [ ]:
# Dividindo os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()

# Treinando o modelo com os dados de treinamento
model.fit(X_train, y_train)


LinearRegression()

In [ ]:
# Fazendo previsões com os dados de teste
y_pred = model.predict(X_test)

# Calculando as métricas de desempenho
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibindo os resultados
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 0.7943225294956073
Mean Squared Error (MSE): 1.7588233889912683
R² Score: 0.4191690491135399


In [ ]:
# Validando o modelo com validação cruzada (cross-validation)
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')

# Exibindo os resultados da validação cruzada
print(f"Validação Cruzada - MSE médio (5 folds): {-np.mean(cv_scores)}")

Validação Cruzada - MSE médio (5 folds): 1.156420863946607


Linear Regression works better than Random Forest, We can try to apply Hiperparameters Tunning to improve the performance

In [ ]:
param_grid = {
    'fit_intercept': [True, False],
}

In [ ]:
model = LinearRegression()


In [ ]:
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')


In [ ]:
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False]},
             scoring='neg_mean_squared_error')

In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test)


In [ ]:

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 0.7968805096704569
Mean Squared Error (MSE): 1.737450959450604
R² Score: 0.42622704518668597


The performace had a small improvement after tunning the hiperparameters, the model can be used but will have a significant error, for a better prediction was suggested to the client to gatter more data from the projects to be able to have sufficient info for a better model.
